In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import pydicom as dcm
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import math
from PIL import Image

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
keras.mixed_precision.set_global_policy('mixed_float16')
%matplotlib inline

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: A100-SXM4-40GB, compute capability 8.0


In [3]:
inception = keras.applications.resnet.ResNet101(include_top=False, input_shape=(512,512,3), weights='imagenet', 
                                                       classes=6)

2021-12-22 01:57:31.871655: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-22 01:57:32.410773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31915 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [4]:
len(inception.layers)

345

In [5]:
def get_center_and_width(dicom):
    return tuple([int(x[0]) if type(x) == dcm.multival.MultiValue else int(x) for x in [dicom.WindowCenter, dicom.WindowWidth]])
def normalize_minmax(img):
    mi, ma = img.min(), img.max()
    return (img - mi) / (ma - mi)

def window_filter(img, center, width, slope, intercept):
    out = np.copy(img)
    out = out*slope + intercept
    lowest_visible = center - width//2
    highest_visible = center + width//2
    
    out[out < lowest_visible] = lowest_visible
    out[out > highest_visible] = highest_visible
    return normalize_minmax(out)
def standardize(img):
    m, std = img.mean(), img.std()
    return (img - m) / std


In [6]:
model = keras.models.Sequential()
model.add(inception)
model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 16, 16, 2048)      42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 6)                 12294     
                                                                 
Total params: 42,670,470
Trainable params: 42,565,126
Non-trainable params: 105,344
_________________________________________________________________


In [8]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics=['binary_accuracy', 
                       keras.metrics.AUC(multi_label=True, num_labels=6, from_logits=False),
                       keras.metrics.Precision(), keras.metrics.Recall()],
             optimizer=keras.optimizers.Adam(learning_rate=1e-4))

cp_callback = keras.callbacks.ModelCheckpoint(filepath='reproduce_training_2/checkpoint.ckpt',
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
# data_tens = tf.convert_to_tensor(data)
# model.predict(data_tens)

In [10]:
# def get_img_tensor(img_path):
#     dicom = dcm.dcmread(img_path, force=True)
    
#     img = dicom.pixel_array
#     center, width = get_center_and_width(dicom)
#     slope, intercept = dicom.RescaleSlope, dicom.RescaleIntercept
#     brain = window_filter(img, 40, 80, slope, intercept)
#     subdural = window_filter(img, 80, 200, slope, intercept)
#     tissue = window_filter(img, 40, 380, slope, intercept)
    
#     return tf.convert_to_tensor(np.stack([brain, subdural, tissue], axis=2), dtype=tf.dtypes.float32)
def get_img_tensor(img_path):
    return tf.convert_to_tensor(np.asarray(Image.open(img_path), dtype=np.float32) / 255.)

In [11]:
png_path = 'rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/'
labels = pd.read_csv('rsna-intracranial-hemorrhage-detection/train_labels.csv')
labels = {l[0]: l[1:].astype(np.int8) for l in labels.to_numpy()}

In [12]:
class RSNASequence(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = [self.y[img_id.split('/')[-1].split('.')[0]] for img_id in batch_x]
        # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return (tf.stack([get_img_tensor(img_path) for img_path in batch_x], axis=0), 
               tf.convert_to_tensor(batch_y))
    
    def on_epoch_end(self):
        ind = np.random.choice(list(range(len(os.listdir(png_path)))), size=train_cutoff, replace=False)
        self.x = [png_path + img_name for img_name in np.array(os.listdir(png_path))[ind]]
    

In [13]:
train_cutoff = 25600 #training the whole dataset takes ~9 hours, so we cut it short for proof-of-concept purposes.
ind = np.random.choice(list(range(len(os.listdir(png_path)))), size=train_cutoff, replace=False)
train_sequence = RSNASequence([png_path + img_name for img_name in np.array(os.listdir(png_path))[ind]], labels, 32)

In [14]:
model.fit(x=train_sequence, epochs=3, callbacks=[cp_callback])
# model.predict(train_sequence[1][0])
# train_sequence[1][0]

Epoch 1/3


2021-12-22 01:58:02.308813: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200


800/800 [==============================] - ETA: 0s - loss: 0.1269 - binary_accuracy: 0.9557 - auc: 0.8826 - precision: 0.7119 - recall: 0.3602
Epoch 00001: saving model to reproduce_training_2/checkpoint.ckpt
800/800 [==============================] - 1140s 1s/step - loss: 0.1269 - binary_accuracy: 0.9557 - auc: 0.8826 - precision: 0.7119 - recall: 0.3602
Epoch 2/3
800/800 [==============================] - ETA: 0s - loss: 0.1006 - binary_accuracy: 0.9646 - auc: 0.9219 - precision: 0.7839 - recall: 0.5086
Epoch 00002: saving model to reproduce_training_2/checkpoint.ckpt
800/800 [==============================] - 1076s 1s/step - loss: 0.1006 - binary_accuracy: 0.9646 - auc: 0.9219 - precision: 0.7839 - recall: 0.5086
Epoch 3/3
228/800 [=======>......................] - ETA: 12:30 - loss: 0.0971 - binary_accuracy: 0.9669 - auc: 0.9256 - precision: 0.8087 - recall: 0.5583

2021-12-22 02:39:38.362445: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: UnidentifiedImageError: cannot identify image file 'rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/ID_5589465c2.png'
Traceback (most recent call last):

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 275, in __call__
    ret = func(*args)

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py", line 649, in wrapper
    return func(*args, **kwargs)

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 992, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 834, in wrapped_generator
    for data in generator_fn():

  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 960, in generator_fn
    yield x[i]

  File "/tmp/ipykernel_5003/1718

UnknownError: 2 root error(s) found.
  (0) UNKNOWN:  UnidentifiedImageError: cannot identify image file 'rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/ID_5589465c2.png'
Traceback (most recent call last):

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 275, in __call__
    ret = func(*args)

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py", line 649, in wrapper
    return func(*args, **kwargs)

  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 992, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 834, in wrapped_generator
    for data in generator_fn():

  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 960, in generator_fn
    yield x[i]

  File "/tmp/ipykernel_5003/1718398186.py", line 13, in __getit

In [15]:
model.save("reproduce_training_2/model.h5")

/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


We can see that training on just a few thousand of the 700k+ DICOM training images yields a 96% accuracy and an AUC of 0.92 on the training dataset, using ResNet101 pretrained weights as a base and fine tuning using a custom softmax binary cross-entropy layer. These preliminary results are very promising, however it is unknown whether the model can generalize to the entire training set let alone the test set. Additionally, the recall of ~0.5 is rather low, reflecting the highly imbalanced nature of the dataset. However, I believe that I have succeeded in reproducing one part of the research that I have conducted- all solutions I found used a CNN with pretrained weights as the first stage for their model. While reproducing the entirety of their research would take up to a week of coding, debugging, and training (not to mention money); I believe that my results are sufficient for this capstone submission (which predicted a 5-20 hour timeframe). 

When testing full models, I plan to start by training at least 1 epoch on the full dataset with the CNN, then expirement with feeding the results and the extracted features (taken from the CNN's pennultimate layer) to feed into:
1. An ensemble method of boosting or bagging tree methods (as one of my sources did)
2. A RNN, perhaps a LSTM
3. Another CNN
4. An ensemble of RNNs and/or CNNs

Such experimentation will take lots of time and I worry that it will bring me above the $300 credit limit alloted to me for Paperspace.